In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import numpy as np 
import pandas as pd 

from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

import tensorflow as tf
import keras
from tensorflow.keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras import models, layers
from keras.applications import resnet50
from keras.models import Model
from keras.layers import Input

import matplotlib.pyplot as plt
import cv2

2024-03-16 13:42:28.090788: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-16 13:42:28.122177: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-16 13:42:28.122208: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-16 13:42:28.122973: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-16 13:42:28.127967: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-16 13:42:28.128493: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
for dirname, _, filenames in os.walk('./data'):
    print(dirname)

./data
./data/Uninfected
./data/Parasitized
./data/cell_images
./data/cell_images/Uninfected
./data/cell_images/Parasitized


In [4]:
def data_prep(parasitized, uninfected):
    
    parasitized = ["./data/Parasitized" + '/' +  parasitize for parasitize in parasitized]
    uninfected = ["./data/Uninfected" + '/' +  uninfect for uninfect in uninfected]

    labels = len(parasitized) * ['parasitized'] + len(uninfected) * ['uninfected']
    data = parasitized + uninfected

    return pd.DataFrame({'Image_Path': data , 'Labels': labels})

In [5]:
df = data_prep(os.listdir("./data/Parasitized/"), os.listdir("./data/Uninfected/"))

In [6]:
print("Shape of the DataFrame:", df.shape)
print("\nHead of the DataFrame:")
print(df.head())
print("\nTail of the DataFrame:")
print(df.tail())

Shape of the DataFrame: (27558, 2)

Head of the DataFrame:
                                          Image_Path       Labels
0  ./data/Parasitized/C137P98ThinF_IMG_20151005_1...  parasitized
1  ./data/Parasitized/C133P94ThinF_IMG_20151004_1...  parasitized
2  ./data/Parasitized/C39P4thinF_original_IMG_201...  parasitized
3  ./data/Parasitized/C70P31_ThinF_IMG_20150819_1...  parasitized
4  ./data/Parasitized/C132P93ThinF_IMG_20151004_1...  parasitized

Tail of the DataFrame:
                                              Image_Path      Labels
27553  ./data/Uninfected/C145P106ThinF_IMG_20151016_1...  uninfected
27554  ./data/Uninfected/C72P33_ThinF_IMG_20150815_10...  uninfected
27555  ./data/Uninfected/C101P62ThinF_IMG_20150918_15...  uninfected
27556  ./data/Uninfected/C7NthinF_IMG_20150611_105444...  uninfected
27557  ./data/Uninfected/C183P144NThinF_IMG_20151201_...  uninfected


In [8]:
from tensorflow.keras.layers import BatchNormalization
# Function to load and preprocess images
def load_images(file_paths, target_size=(64, 64)):
    images = []
    for path in file_paths:
        img = cv2.imread(path)
        img = cv2.resize(img, target_size)
        images.append(img)
    return np.array(images)

# Load and preprocess images
X = load_images(df['Image_Path'])
y = df['Labels']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Build an improved CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())  # Adding Batch Normalization to stabilize and accelerate the training process
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(256, activation='relu'))  # Increased the number of neurons in the fully connected layer
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))  # Additional dense layer
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))  # Additional dense layer
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the improved CNN model
model.fit(X_train, (y_train == 'parasitized').astype(int), epochs=10, batch_size=32, validation_split=0.2)


# Extract features from the CNN model
cnn_features_train = model.predict(X_train)
cnn_features_test = model.predict(X_test)

# Reshape features for KNN
cnn_features_train = cnn_features_train.reshape(cnn_features_train.shape[0], -1)
cnn_features_test = cnn_features_test.reshape(cnn_features_test.shape[0], -1)

# Build a KNN model
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(cnn_features_train, (y_train == 'parasitized'))

# Predict using the hybrid model
knn_predictions = knn_model.predict(cnn_features_test)

# Evaluate the hybrid model
accuracy = accuracy_score((y_test == 'parasitized'), knn_predictions)
print()
print(f"Accuracy of the hybrid model: {accuracy*100}")

Epoch 1/10
552/552 [==============================] - 40s 70ms/step - loss: 0.4362 - accuracy: 0.8095 - val_loss: 0.1897 - val_accuracy: 0.9354
Epoch 2/10
552/552 [==============================] - 39s 71ms/step - loss: 0.1913 - accuracy: 0.9371 - val_loss: 0.1584 - val_accuracy: 0.9528
Epoch 3/10
552/552 [==============================] - 43s 78ms/step - loss: 0.1722 - accuracy: 0.9462 - val_loss: 0.1486 - val_accuracy: 0.9567
Epoch 4/10
552/552 [==============================] - 41s 73ms/step - loss: 0.1544 - accuracy: 0.9518 - val_loss: 0.1597 - val_accuracy: 0.9553
Epoch 5/10
552/552 [==============================] - 41s 74ms/step - loss: 0.1427 - accuracy: 0.9541 - val_loss: 0.1277 - val_accuracy: 0.9583
Epoch 6/10
552/552 [==============================] - 41s 74ms/step - loss: 0.1266 - accuracy: 0.9580 - val_loss: 0.1167 - val_accuracy: 0.9615
Epoch 7/10
552/552 [==============================] - 40s 73ms/step - loss: 0.1257 - accuracy: 0.9591 - val_loss: 0.1350 - val_accuracy: